## 📈 Stock Predictor - Transformer Model (Darts)
This notebook trains a Transformer model using the `darts` library to predict the next-day closing price.

In [ ]:
# Parameters
ticker = "AAPL"

In [ ]:
import pandas as pd
from src.data.fetcher import fetch_data
from darts.models import TransformerModel
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse
import matplotlib.pyplot as plt
import os

In [ ]:
# Fetch full historical data
df = fetch_data(ticker, period="max")
df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date'])
df['Date'] = df['Date'].dt.tz_localize(None)  # ✅ Strip timezone
print('✅ Final dtype of Date:', df['Date'].dtype)

In [ ]:
# Convert to TimeSeries
series = TimeSeries.from_dataframe(
    df,
    time_col="Date",
    value_cols="Close",
    fill_missing_dates=True,
    freq="D"
)
scaler = Scaler()
series_scaled = scaler.fit_transform(series)

In [ ]:
# Train-validation split
train, val = series_scaled.split_before(0.8)

In [ ]:
# Define Transformer model
model = TransformerModel(
    input_chunk_length=30,
    output_chunk_length=1,
    n_epochs=100,
    model_name=f"transformer_{ticker}",
    random_state=42,
    save_checkpoints=True,
    force_reset=True
)

In [ ]:
# Train the model
model.fit(train, verbose=True)

In [ ]:
# Forecast
forecast = model.predict(n=len(val))
forecast_actual = scaler.inverse_transform(forecast)
val_actual = scaler.inverse_transform(val)

In [ ]:
# Evaluate
error = rmse(val_actual, forecast_actual)
print(f"Test RMSE: {error:.2f}")

In [ ]:
# Plot
plt.figure(figsize=(10,6))
val_actual.plot(label="Actual")
forecast_actual.plot(label="Predicted")
plt.title(f"{ticker} - Actual vs Predicted")
plt.legend()
os.makedirs(f"artifacts/reports", exist_ok=True)
plt.savefig(f"artifacts/reports/{ticker}_actual_vs_predicted.png")
plt.show()

In [ ]:
# Save model and scaler
import joblib 

os.makedirs("artifacts/models", exist_ok=True)
model.save(f"artifacts/models/transformer_{ticker}.pt")
scaler.save(f"artifacts/models/scaler_{ticker}.pkl")